In [ ]:
# LLM getter
from model import get_conversation
# Veri seti oluşturucu LLM model (LLama-450B)



In [ ]:
# Tekli Link getter
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status() 
        soup = BeautifulSoup(response.content, 'html.parser')
        
        links = soup.find_all('a')
        
        urls = [urljoin(url, link.get('href')) for link in links if link.get('href') is not None]
        
        return urls
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return []

url = 'https://www.webtekno.com/'
url = 'https://www.webtekno.com/kripto-para'
url = 'https://www.webtekno.com/iflas-eden-kripto-para-borsalari-h146330.html'
url = 'https://finansalokuryazarlik.spl.com.tr/'
url = 'https://www.parasut.com/blog/finansal-okuryazarlik-nedir-ve-nasil-gelistirilir'

links = list(set(get_links(url)))
print(links)


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# Recursive text getter

def extract_long_segments(text, threshold=2):
    # Split the text into segments using \n as the delimiter
    segments = text.split('\n')
    
    # Initialize an empty list to hold the valid segments
    long_segments = []
    
    for segment in segments:
        # Split each segment into sentences
        sentences = segment.split('.')
        # Count the number of sentences with at least one word
        sentence_count = sum(1 for sentence in sentences if sentence.strip())
        
        # If the number of sentences is greater than or equal to the threshold, add the segment to the result 
        if sentence_count >= threshold:
            long_segments.append(segment)
    
    return long_segments
    
def get_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        links = soup.find_all('a')
        urls = [urljoin(url, link.get('href')) for link in links if link.get('href') is not None]
        
        return urls
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return []

def get_text(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        text = soup.get_text(separator='\n', strip=True)
        
        return text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return ''

def get_all_links(url, depth):
    all_links = set()

    def fetch_links(current_url, current_depth):
        if current_depth > depth:
            return
        links = get_links(current_url)
        for link in links:
            if link not in all_links:
                all_links.add(link)
                fetch_links(link, current_depth + 1)

    fetch_links(url, 1)
    return all_links

def get_all_texts(url, depth):
    all_links = get_all_links(url, depth)
    all_texts = [get_text(link) for link in all_links]
    return all_texts


def split_text_into_chunks(text, chunk_size):
    """
    Uzun bir metni kelimeleri bölmeden daha küçük parçalara böler.

    ilgili parametreler:
    
    text (str): Bölünecek uzun metin.
    chunk_size (int): Her bir parçanın maksimum boyutu.
    Döndürür:
    List[str]: Metin parçalarından oluşan bir liste.
    
    """
    
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        
        if len(' '.join(current_chunk + [word])) > chunk_size:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
        else:
            current_chunk.append(word)

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks


url = 'https://finansalokuryazarlik.spl.com.tr/'
#url = 'https://www.fo-der.org/finansal-okuryazarlik-nedir/'

depth = 1
all_texts = "".join(get_all_texts(url, depth))
print(all_texts)


In [35]:
file_name = "./finansalokuryazarlik"
os.makedirs(file_name+'_json/', exist_ok=True)



In [ ]:
from model import *

web_scrape_command = """
    Verilen text'den Türkçe Dil modeli eğitim seti oluştur. Instruction1, Respnse1, Instruction2, Response2 .. .. formatında olsun ve Türkçe dilinde yaz.
    Örnek bir çift:
    Instruction1: Bir kişi, birikimlerini değerlendirmek için çeşitli yatırım araçları arasında hangisinin daha uygun olduğunu belirlemeye çalışıyor. Bu kişi, yüksek getiri potansiyeli ile birlikte risk seviyelerini de göz önünde bulundurarak nasıl bir yatırım stratejisi oluşturmalıdır?
    
    Response1: Risk ve Getiri Dengesini Anlamak:
    Borsa: Yüksek getiri potansiyeline sahip olmakla birlikte, yüksek volatilite (dalgalanma) nedeniyle riski de yüksektir. 
    Devlet Tahvilleri: Daha düşük risk taşır ve sabit getiri sağlar. Özellikle riskten kaçınan ve sermayesini korumak isteyen yatırımcılar için uygundur. Ancak getirisi, borsa veya diğer riskli yatırımlara göre genellikle daha düşüktür.

    """
conversation = get_conversation()

chunk_size = 2000

chunks = split_text_into_chunks(text, chunk_size)
for i, chunk in enumerate(chunks):
    response = conversation.predict(human_input=web_scrape_command + chunk)
    json_data = json.dumps(extract_data(response), ensure_ascii=False, indent=4)
            
    with open(file_name+ str(i)+".json", 'w', encoding='utf-8') as f:
        f.write(json_data)
        